In [1]:
import pandas as pd
import numpy as np
import scipy
import sklearn
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split ##split the data
from sklearn.feature_selection import RFE ##Test Features and select the best ones
from sklearn.linear_model import LogisticRegression ##Test Features and select the best ones
import statsmodels.api as sm ##Stats
from sklearn.metrics import confusion_matrix ##F1 score
from sklearn import preprocessing ##Normalize feature
import missingno as msno 
import datetime as dt
from itertools import product
from pandas_profiling import ProfileReport
from datetime import datetime
import calendar
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
## Reading CSV
donors = pd.read_csv("donors.csv", low_memory= False)

In [3]:
donors_id = donors.loc[:,'ODATEDW':'PEPSTRFL']

In [4]:
nan_count = donors_id.isnull().sum().sort_values(ascending = False)

In [5]:
nan_count = nan_count[nan_count != 0]
nan_count

NUMCHLD     83026
MBCOLECT    52914
PUBDOITY    52854
PUBGARDN    52854
PUBHLTH     52854
MAGMALE     52854
PUBNEWFN    52854
PUBPHOTO    52854
PUBOPP      52854
MAGFEM      52854
MAGFAML     52854
MBBOOKS     52854
MBGARDEN    52854
MBCRAFT     52854
PUBCULIN    52854
WEALTH1     44732
WEALTH2     43823
DOB         23883
INCOME      21286
dtype: int64

In [6]:
nan_count.index

Index(['NUMCHLD', 'MBCOLECT', 'PUBDOITY', 'PUBGARDN', 'PUBHLTH', 'MAGMALE',
       'PUBNEWFN', 'PUBPHOTO', 'PUBOPP', 'MAGFEM', 'MAGFAML', 'MBBOOKS',
       'MBGARDEN', 'MBCRAFT', 'PUBCULIN', 'WEALTH1', 'WEALTH2', 'DOB',
       'INCOME'],
      dtype='object')

In [7]:
#Delete Columns with more than 50% of Nan
donors_id_r1 = donors_id.drop(columns = \
                              ['MBCOLECT','PUBDOITY', 'PUBGARDN', 'PUBHLTH', 'MAGMALE','PUBNEWFN', 
                               'PUBPHOTO', 'PUBOPP', 'MAGFEM', 'MAGFAML', 'MBBOOKS','MBGARDEN', 
                               'MBCRAFT', 'PUBCULIN'])

In [8]:
nan_count_r1 = donors_id_r1.isnull().sum().sort_values(ascending = False)

In [9]:
nan_count_r1 = nan_count_r1[nan_count_r1 != 0]
nan_count_r1

NUMCHLD    83026
WEALTH1    44732
WEALTH2    43823
DOB        23883
INCOME     21286
dtype: int64

In [10]:
#Correting the variables to datetime types.
donors_id_r1['ODATEDW'] = donors_id_r1['ODATEDW'].map(lambda x: datetime.strptime(x, '%Y-%m-%d') if pd.notnull(x) else x)
donors_id_r1['DOB'] = donors_id_r1['DOB'].map(lambda x: datetime.strptime(x, '%Y-%m-%d') if pd.notnull(x) else x)

In [11]:
#Create Variable AGE
donors_id_r1['AGE'] = datetime.today().year - donors_id_r1['DOB'].dt.year

# Separate the variables
# Financial information
- INCOME                      HOUSEHOLD INCOME -> **'Ordinary''KNN IMPUTER'**
- WEALTH1                     Wealth Rating -> **'Ordinary' 'PCA/KNN IMPUTER'**    
- WEALTH2                     Wealth Rating -> **'Ordinary' 'PCA/KNN IMPUTER'**
# Geographic information
- STATE                       State abbreviation (a nominal/symbolic field) -> **'Discretizing'**
- ZIP                         Zipcode (a nominal/symbolic field) -> **'Delete'**
- MAILCODE                    Mail Code -> **'Binary'**
- PVASTATE                    EPVA State or PVA State -> **'Binary'**
- DOMAIN                      DOMAIN/Cluster code. A nominal or symbolic field. -> **'Separate in 2'**
- GEOCODE                     Geo Cluster Code indicating the level geography at which -> **' Ordinary'**
# Donation History
- OSOURCE                     Origin Source -> **'DELETE'**
- MDMAUD                      The Major Donor Matrix code -> **'Category'**
- MAJOR                       Major Donor Flag -> **'Binary'**
- PEPSTRFL                    Indicates PEP Star RFA Status -> **'Binary'**
- HIT                         MOR Flag HIT (Mail Order Response) -> **'Metric'**
# ID information
- ODATEDW                     Origin Date. Date of donors first gift -> **'DELETE'**
- TCODE                       Donor title code -> 'DELETE'
- DOB                         Date of birth (YYMM, Year/Month format.) -> **'Create Age' and 'DELETE'**
- CHILD03                     Presence of Children age 0-3 -> **'Fill Nan with 0'**
- CHILD07                     Presence of Childern age 4-7 -> **'Fill Nan with 0'**
- CHILD12                     Presence of Childern age 8-12 -> **'Fill Nan with 0'**
- CHILD18                     Presence of Childern age 13-18 -> **'Fill N-> an with 0'**
- NUMCHLD                     NUMBER OF CHILDREN -> **'DELETE'**
- GENDER                      Gender -> **'Fill Nan with U'**
- DATASRCE                    Source of Overlay Data -> **'DELETE'**
- MALEMILI                    % Males active in the Military -> **'Metric'**
- MALEVET                     % Males Veterans -> **'Metric'**
- VIETVETS                    % Vietnam Vets -> **'Metric'**
- WWIIVETS                    % WWII Vets -> **'Metric'**
- LOCALGOV                    % Employed by Local Gov -> **'Metric'**
- STATEGOV                    % Employed by State Gov -> **'Metric'**
- FEDGOV                      % Employed by Fed Gov -> **'Metric'**
- COLLECT1                    COLLECTABLE (Y/N) -> **'Binary'**
- VETERANS                    VETERANS (Y/N) -> **'Binary'**
- BIBLE                       BIBLE READING (Y/N) -> **'Binary'**
- CATLG                       SHOP BY CATALOG (Y/N) -> **'Binary'**
- HOMEE                       WORK FROM HOME (Y/N) -> **'Binary'**
- PETS                        HOUSEHOLD PETS (Y/N) -> **'Binary'**
- CDPLAY                      CD PLAYER OWNERS (Y/N) -> **'Binary'**
- STEREO                      STEREO/RECORDS/TAPES/CD (Y/N) -> **'Binary'**
- PCOWNERS                    HOME PC OWNERS/USERS -> **'Binary'**
- PHOTO                       PHOTOGRAPHY (Y/N) -> **'Binary'**
- CRAFTS                      CRAFTS (Y/N) -> **'Binary'**
- FISHER                      FISHING (Y/N) -> **'Binary'**
- GARDENIN                    GARDENING (Y/N) -> **'Binary'**
- BOATS                       POWER BOATING (Y/N) -> **'Binary'**
- WALKER                      WALK FOR HEALTH (Y/N) -> **'Binary'**
- KIDSTUFF                    BUYS CHILDRENS PRODUCTS (Y/N) -> **'Binary'**
- CARDS                       STATIONARY/CARDS BUYER (Y/N) -> **'Binary'**
- PLATES                      PLATE COLLECTOR (Y/N) -> **'Binary'**
- LIFESRC                     LIFE STYLE DATA SOURCE/ SAME AS DATASRCE -> **'DELETE'**
- HOMEOWNR                    Home Owner Flag -> **'Ordinary'**

# Data permission for ID information
- NOEXCH                      Do Not Exchange Flag (For list rental) -> **'Binary, need to fix the labels'**
- RECINHSE                    In House File Flag -> **'Binary'**
- RECP3                       P3 File Flag -> **'Binary'**
- RECPGVG                     Planned Giving File Flag -> **'Binary'**
- RECSWEEP                    Sweepstakes file fla g -> **'Binary'**
# Unknown 
- SOLP3                       SOLICIT LIMITATION CODE P3 -> **'Ordinary'**
- SOLIH                       SOLICITATION LIMIT CODE IN HOUSE -> **'Ordinary'**

In [12]:
#Drop Columns ZIP ODATEDW TCODE DOB DATASRCE LIFESRC
donors_id_r1 = donors_id_r1.drop(columns = ['STATE', 'OSOURCE', 'ZIP', 'NUMCHLD', 'ODATEDW', 'TCODE', 'DOB', 'DATASRCE', 'LIFESRC'])

In [13]:
#Fill ' ' values to Nan to fill with KNNImputer
donors_id_r1['DOMAIN'][donors_id_r1['DOMAIN'] == ' '] = None

C:\Users\adria\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
#Split Variable DOMAIN wealth and type of neighbor.
donors_id_r1['DOMAIN_LVL'] = donors_id_r1['DOMAIN'].str.split('', expand = True)[1]
donors_id_r1['DOMAIN_ECO'] = donors_id_r1['DOMAIN'].str.split('', expand = True)[2]

In [15]:
#One hot encoder, using dummy_na = True to not descart the missing values
DOMAIN_LVL_OHE = pd.get_dummies(donors_id_r1['DOMAIN_LVL'], dummy_na = True)

In [16]:
#Binary Feature using drop_first to have just one column.
MAJOR_OHE = pd.get_dummies(donors_id_r1['MAJOR'], drop_first = True)

In [17]:
#Binary Feature using drop_first to have just one column.
PEPSTRFL_OHE = pd.get_dummies(donors_id_r1['PEPSTRFL'], drop_first = True)

In [18]:
#Binary Feature using drop_first to have just one column.
ID_binary = pd.get_dummies(donors_id_r1.loc[:,'COLLECT1':'PLATES'], drop_first = True)

In [19]:
nan_count_r1 = donors_id_r1.isnull().sum().sort_values(ascending = False)
nan_count_r1 = nan_count_r1[nan_count_r1 != 0]
nan_count_r1

WEALTH1       44732
WEALTH2       43823
AGE           23883
INCOME        21286
DOMAIN_ECO     2316
DOMAIN         2316
DOMAIN_LVL     2316
dtype: int64

In [20]:
#Transform LASTDATE to create Recency
donors['LASTDATE'] = donors['LASTDATE'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [21]:
#Create Recency
#Last promotion date
lastpromotiondate = datetime.strptime('2017-06-01', '%Y-%m-%d')
lastpromotiondate.year
#Date associated with the most recent gift
donors['RECENCY'] = (lastpromotiondate.year - donors['LASTDATE']
                     .dt.year)*12 + (lastpromotiondate.month - donors['LASTDATE']
                                                                              .dt.month)

In [22]:
#Selected Variables to fill Missing Values:
fill_na_id = pd.concat([donors_id_r1[['AGE', 'WEALTH1', 'WEALTH2', 'INCOME','DOMAIN_ECO']],
                      donors[['RECENCY', 'NGIFTALL', 'AVGGIFT']],
                      DOMAIN_LVL_OHE,
                      MAJOR_OHE,
                      PEPSTRFL_OHE,
                      ID_binary], axis = 1)

In [23]:
#Using KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")
fill_na_id_imputer = imputer.fit_transform(fill_na_id)

In [24]:
fill_na_id[:] = fill_na_id_imputer

KeyError: nan

In [ ]:
fill_na_id.columns

In [ ]:
#Round values WEALTH1, WEALTH2, AGE, INCOME, DOMAIN_ECO, DOMAIN_LVL, DOMAIN to not have float.

fill_na_id[['WEALTH1', 'WEALTH2', 'AGE', 'INCOME', 'DOMAIN_ECO']] = fill_na_id[['WEALTH1', 
                                                                                'WEALTH2', 
                                                                                'AGE', 
                                                                                'INCOME', 
                                                                                'DOMAIN_ECO']].round()

In [ ]:
#Replace Variables, without Domain because the best way to use categorical + ordinal feature is leave the Nan values.
na_columns = ['AGE', 'WEALTH1', 'WEALTH2', 'INCOME']
donors_id_r1[na_columns] = fill_na_id[na_columns]

In [ ]:
#Discretizing the Age feature:
labels = ["Child", "Teenager", "Adult", "Senior"]
bins = pd.IntervalIndex.from_tuples([(0, 12), (12, 19), (19, 59), (59, 200)])
donors_id_r1['AGE_Group'] = pd.cut(donors_id_r1['AGE'], bins = bins).map(dict(zip(bins, labels)))

In [ ]:
#Filling NaN values from DOMAIN to One Hot Encode
donors_id_r1[['DOMAIN','DOMAIN_LVL','DOMAIN_ECO']] = donors_id_r1[['DOMAIN','DOMAIN_LVL','DOMAIN_ECO']].fillna('')

In [ ]:
nonmetric_features = donors_id_r1.dtypes[(donors_id_r1.dtypes == 'object') | (donors_id_r1.dtypes == 'category')].index.to_list()

In [ ]:
metric_features = donors_id_r1.dtypes[(donors_id_r1.dtypes != 'object') & (donors_id_r1.dtypes != 'category')].index.to_list()

In [ ]:
train_data_standard_ohe = donors_id_r1.copy()
ohc = OneHotEncoder(sparse=False, drop = 'if_binary')
ohc_feat = ohc.fit_transform(train_data_standard_ohe[nonmetric_features])
ohc_feat_names = ohc.get_feature_names()
ohc_df = pd.DataFrame(ohc_feat, index=train_data_standard_ohe[nonmetric_features].index, columns=ohc_feat_names)  # Why the index=df_ohc.index?
ohc_df


In [ ]:
donors_cluster = pd.concat([donors_id_r1[metric_features],
                            ohc_df],
                           axis = 1) 

In [ ]:
donors_cluster.head()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
kmeans = KMeans(init='k-means++', n_clusters=8, n_init=10, max_iter=300)

In [ ]:
kmeans.fit(donors_cluster)

In [ ]:
kmeans.n_iter_

In [ ]:
donors_cluster['Cluster'] = kmeans.labels_

In [ ]:
donors_cluster.groupby(['Cluster']).agg(['mean'])

In [ ]:
donors_census = donors.loc[:,'POP901':'AC2']

In [ ]:
nan_count_census = donors_census.isnull().sum().sort_values(ascending = False)

In [ ]:
nan_count_census = nan_count_census[nan_count_census != 0]
nan_count_census

In [ ]:
donors_census['DMA'].value_counts()

In [ ]:
donors_census['ADI']

In [ ]:
donors_promotion = donors.loc[:,'ADATE_2':'GEOCODE2']

In [ ]:
nan_count_promotion = donors_promotion.isnull().sum().sort_values(ascending = False)

In [ ]:
nan_count_promotion = nan_count_promotion[nan_count_promotion != 0]
nan_count_promotion

In [ ]:
RDATE = nan_count_promotion[nan_count_promotion.index.str.contains("RDATE_")].index
RAMNT = nan_count_promotion[nan_count_promotion.index.str.contains("RAMNT_")].index
ADATE = nan_count_promotion[nan_count_promotion.index.str.contains("ADATE_")].index
nan_count_promotion = nan_count_promotion.drop(labels = RDATE)
nan_count_promotion = nan_count_promotion.drop(labels = RAMNT)
nan_count_promotion = nan_count_promotion.drop(labels = ADATE)

In [ ]:
nan_count_promotion

In [ ]:
donors_promotion[donors_promotion['TIMELAG'].isnull()]['NGIFTALL'].unique()

In [ ]:
donors_promotion[donors_promotion['FISTDATE'].isnull()]['NGIFTALL']